In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with BigQuery ML Training

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fbigquery_ml%2Fget_started_with_bqml_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/bigquery_ml/get_started_with_bqml_training.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
     </a>
   </td>
</table>
<br/><br/><br/>

## Overview

This tutorial demonstrates how to use Vertex AI in production. This tutorial covers get started with BigQuery ML training.

Learn more about [BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### Objective

In this tutorial, you learn how to use BigQuery ML for training with Vertex AI.

This tutorial uses the following Google Cloud ML services:

- BigQuery ML training
- Vertex AI model resource
- Vertex AI Vizier

The steps performed include:

- Create a local BigQuery table in your project
- Train a BigQuery ML model
- Evaluate the BigQuery ML model
- Export the BigQuery ML model as a cloud model
- Upload the exported model as a Vertex AI model resource
- Hyperparameter tune a BigQuery ML model with Vertex AI Vizier
- Automatically register a BigQuery ML model to Vertex AI Model Registry

### Dataset

The dataset used for this tutorial is the Penguins dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). This version of the dataset is used to predict the species of penguins from the available features like culmen-length, flipper-depth etc.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage
- BigQuery

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[Cloud Storage pricing](https://cloud.google.com/storage/pricing) and
[BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the 
[Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started
Install Vertex AI SDK for Python and other required packages

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet pyarrow \
                                 google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 google-cloud-bigquery-storage \
                                 db-dtypes 

### Restart runtime (Colab only)
To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">,
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>,
</div>

### Authenticate your notebook environment (Colab only)
Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information
Learn more about [setting up a project and a development environment.](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} {BUCKET_URI}

#### Service Account

You use a service account to create Vertex AI Pipeline jobs. If you don't want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python and BigQuery SDK  with your project and the created bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### Set hardware accelerators

You can set hardware accelerators for prediction.

Set the variable `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa T4 GPUs allocated to each VM, you would specify:

    (aiplatform.AcceleratorType.NVIDIA_TESLA_T4, 4)

Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [about hardware accelerator support for your region.](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

In [ ]:
import os

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)

### Set prebuilt containers

Set the prebuilt Docker container image for prediction.

- Set the variable `TF` to the TensorFlow version of the container image. The following list shows some of the prebuilt images available:


For the latest list, see [prebuilt containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers).

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

### Set machine type

Next, set the machine type to use for prediction.

- Set the variable `DEPLOY_COMPUTE` to configure the compute resources for the VM which is used for prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: You may also use n2 and e2 machine types for training and deployment, but they don't support GPUs*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## BigQuery ML introduction

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification and regression in BigQuery using SQL syntax.

Learn more about [BigQuery ML documentation](https://cloud.google.com/bigquery-ml/docs).

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### Create BQ dataset resource

First, you create an empty dataset resource in your project.

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}
"""

job = bqclient.query(DATASET_QUERY)

### Train BigQuery ML model

Next, you create and train a BigQuery ML tabular classification model from the public dataset penguins and store the model in your project using the `CREATE MODEL` statement. The model configuration is specified in the `OPTIONS` statement as follows:

- `model_type`: The type and archictecture of tabular model to train, e.g., DNN classification.
- `labels`: The column labels.

Learn more about [The CREATE MODEL statement](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create).

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species']
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### Evaluate the trained BigQuery ML model

Next, retrieve the model evaluation for the trained BigQuery ML model.

Learn more about [The ML.EVALUATE function](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate).

In [ ]:
EVAL_QUERY = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
ORDER BY  roc_auc desc
LIMIT 1"""

try:
    job = bqclient.query(EVAL_QUERY)
    results = job.result().to_dataframe()
    print(results)
except Exception as e:
    print(e)

### Export the model from BigQuery ML

The model you trained in BigQuery ML is a TensorFlow model. Next, export the TensorFlow model artifacts in TF.SavedModel format.

In [ ]:
param = f"{PROJECT_ID}:{BQ_DATASET_NAME}.{MODEL_NAME} {BUCKET_URI}/{MODEL_NAME}"
! bq extract -m $param

MODEL_DIR = f"{BUCKET_URI}/{BQ_DATASET_NAME}"
! gsutil ls $MODEL_DIR

## Upload the BigQuery ML model to a Vertex AI Model resource

Finally, now that you have the BigQuery ML model exported, you upload the model artifacts to Vertex AI model resource, in the same way as if you were uploading a custom trained model.

Below is a partial list of mapping BigQuery ML model types to their corresponding exported model format:

'LINEAR_REG'<br/>
'LOGISTIC_REG' --> TensorFlow SavedFormat

'AUTOML_CLASSIFIER'<br/>
'AUTOML_REGRESSOR' --> TensorFlow SavedFormat

'BOOSTED_TREE_CLASSIFIER'<br/>
'BOOSTED_TREE_REGRESSOR' --> XGBoost format

'DNN_CLASSIFIER'<br/>
'DNN_REGRESSOR'<br/>
'DNN_LINEAR_COMBINED_CLASSIFIER'<br/>
'DNN_LINEAR_COMBINED_REGRESSOR' --> TensorFlow Estimator

In [ ]:
try:
    model = aiplatform.Model.upload(
        display_name="penguins",
        artifact_uri=MODEL_DIR,
        serving_container_image_uri=DEPLOY_IMAGE,
        sync=True,
    )
except Exception as e:
    print(e)

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, invoke the `deploy` method, with the following parameters:

- `deployed_model_display_name`: A human readable name for the deployed model.
- `traffic_split`: Percent of traffic at the endpoint that goes to this model, which is specified as a dictionary of one or more key/value pairs.
If only one model, then specify as { "0": 100 }, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
If there are existing models on the endpoint, for which the traffic needs to split, then use model_id to specify as { "0": percent, model_id: percent, ... }, where model_id is the model id of an existing model to the deployed endpoint. The percent must add up to 100.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

In [ ]:
DEPLOYED_NAME = "penguins"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

try:
    if DEPLOY_GPU:
        endpoint = model.deploy(
            deployed_model_display_name=DEPLOYED_NAME,
            traffic_split=TRAFFIC_SPLIT,
            machine_type=DEPLOY_COMPUTE,
            accelerator_type=DEPLOY_GPU.name,
            accelerator_count=DEPLOY_NGPU,
            min_replica_count=MIN_NODES,
            max_replica_count=MAX_NODES,
        )
    else:
        endpoint = model.deploy(
            deployed_model_display_name=DEPLOYED_NAME,
            traffic_split=TRAFFIC_SPLIT,
            machine_type=DEPLOY_COMPUTE,
            min_replica_count=MIN_NODES,
            max_replica_count=MAX_NODES,
        )
except Exception as e:
    print(e)

#### Undeploy the model

When you are done doing predictions, you undeploy the model from the endpoint resource. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
try:
    endpoint.undeploy_all()
except Exception as e:
    print(e)

#### Delete the Vertex AI model resource

The method `delete()` deletes the model.

In [ ]:
try:
    model.delete()
except Exception as e:
    print(e)

### Delete the BigQuery ML model

Next, delete the BigQuery ML instance of the model.

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

try:
    job = bqclient.query(MODEL_QUERY)
except Exception as e:
    print(e)

### Hyperparameter Tune and train a BigQuery ML model

Next, you train a BigQuery ML tabular classification model with hyperparameter tuning using the Vertex AI Vizier service. The hyperparameter settings are specified in the `OPTIONS` statement as follows:

- `HPARAM_TUNING_ALGORITHM`: The algorithm for selecting the next trial parameters.
- `num_trials`: The number of trials.
- `max_parallel_trials`: The number of trials to do in parallel.

Learn more about [Hyperparameter tuning for CREATE MODEL statements](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning).

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    num_trials=10,
    max_parallel_trials=2,
    HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### Evaluate the BigQuery ML trained model

Next, retrieve the model evaluation results for the trained BigQuery ML model.

Learn more about [The ML.EVALUATE function](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate).

In [ ]:
try:
    EVAL_QUERY = f"""
    SELECT *
    FROM
      ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
    ORDER BY  roc_auc desc
    LIMIT 1"""

    job = bqclient.query(EVAL_QUERY)
    results = job.result().to_dataframe()
    print(results)
except Exception as e:
    print(e)

### Delete the BigQuery ML model

Next, delete the BigQuery ML instance of the model.

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

### Train a BigQuery ML model with Explainability

Next, you train the same BigQuery ML model, this time you enable Vertex Explainable AI on the model predictions by adding the option:

- `ENABLE_GLOBAL_EXPLAIN`

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    ENABLE_GLOBAL_EXPLAIN = True
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### Delete the BigQuery ML model

Next, delete the BigQuery ML instance of the model.

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

In [ ]:
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT --role=roles/aiplatform.admin --condition=None

### Training and registering the model

Next, you train the model and automatically register the model to the Vertex AI Model Registry, by adding the following parameters as options:

- `model_registry`: Set to `vertex_ai` to indicate automatic registration to Vertex AI Model Registry.
- `vertex_ai_model_id`: The human readable display name for the registered model.
- `vertex_ai_model_version_aliases`: Alternate name for the model.

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    model_registry="vertex_ai",
    vertex_ai_model_id="bqml_model", 
    vertex_ai_model_version_aliases=["1"]
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

try:
    tblname = job.ddl_target_table
    tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
    print("{} created in {}".format(tblname, job.ended - job.started))
except Exception as e:
    print(e)

### Find the model in the Vertex AI Model Registry

Finally, you can use the Vertex AI model `list()` method with a filter query to find the automatically registered model.

In [ ]:
models = aiplatform.Model.list(filter="display_name=bqml_model")
model = models[0]

print(model.gca_resource)

In [ ]:
models = aiplatform.Model.list()
for model in models:
    if model.gca_resource.display_name.startswith("bqml"):
        print(model.gca_resource.display_name)

### Delete the BigQuery ML model

Next, delete the BigQuery ML instance of the model.

In [ ]:
MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

Set `delete_storage` to `True` to delete the Cloud Storage bucket used in this notebook.

In [ ]:
# Delete the endpoint using the Vertex endpoint object
try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

# Delete the model using the Vertex model object
try:
    model.delete()
except Exception as e:
    print(e)

# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME

delete_storage = False
if delete_storage:
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI